<a href="https://colab.research.google.com/github/ujjawalcse/My-ML-Project/blob/master/Scene_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git

Cloning into 'Dropbox-Uploader'...
remote: Enumerating objects: 951, done.
remote: Total 951 (delta 0), reused 0 (delta 0), pack-reused 951
Receiving objects: 100% (951/951), 309.20 KiB | 568.00 KiB/s, done.
Resolving deltas: 100% (505/505), done.


In [0]:
cd Dropbox-Uploader

/content/Dropbox-Uploader


In [0]:
!chmod +x dropbox_uploader.sh

In [0]:
!bash dropbox_uploader.sh


 This is the first time you run this script, please follow the instructions:

 1) Open the following URL in your Browser, and log in using your account: https://www.dropbox.com/developers/apps
 2) Click on "Create App", then select "Dropbox API app"
 3) Now go on with the configuration, choosing the app permissions and access restrictions to your DropBox folder
 4) Enter the "App Name" that you prefer (e.g. MyUploader57702324324789)

 Now, click on the "Create App" button.

 When your new App is successfully created, please click on the Generate button
 under the 'Generated access token' section, then execute the following command:

 echo "INPUT_YOUR_ACCESS_TOKEN_HERE" > token.txt



In [0]:
!echo "I8vRHfC8loAAAAAAAAAAG6nT7qazIqPxTOww3BMRZh9ZeP5PEoen9NQTzvovoQWV" > token.txt

In [0]:
!bash dropbox_uploader.sh


 This is the first time you run this script, please follow the instructions:

 1) Open the following URL in your Browser, and log in using your account: https://www.dropbox.com/developers/apps
 2) Click on "Create App", then select "Dropbox API app"
 3) Now go on with the configuration, choosing the app permissions and access restrictions to your DropBox folder
 4) Enter the "App Name" that you prefer (e.g. MyUploader84717972197)

 Now, click on the "Create App" button.

 When your new App is successfully created, please click on the Generate button
 under the 'Generated access token' section, then execute the following command:

 echo "INPUT_YOUR_ACCESS_TOKEN_HERE" > token.txt


 Found token.txt with access token: I8vRHfC8loAAAAAAAAAAG6nT7qazIqPxTOww3BMRZh9ZeP5PEoen9NQTzvovoQWV

 You can unlink this account if the token is wrong, using:
 ./dropbox-uploader.sh unlink

 The configuration has been saved.


In [0]:
!bash dropbox_uploader.sh download scene_images.zip

 > Downloading "/scene_images.zip" to "/content/Dropbox-Uploader/scene_images.zip"... DONE


In [0]:
from zipfile import ZipFile
file_name='scene_images.zip'
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.regularizers import l2
from imutils import paths
from tqdm import tqdm
import cv2
import os

Using TensorFlow backend.


In [0]:
train_info=pd.read_csv('scene_images/train.csv')
test_info=pd.read_csv('scene_images/test.csv')
print(train_info.head())
print(test_info.head())
print("Number of labels:",train_info['label'].nunique())

  image_name  label
0      0.jpg      0
1      1.jpg      4
2      2.jpg      5
3      4.jpg      0
4      7.jpg      4
  image_name
0      3.jpg
1      5.jpg
2      6.jpg
3     11.jpg
4     14.jpg
Number of labels: 6


In [0]:
target=pd.Series(train_info['label'])
encoded=pd.get_dummies(target,sparse=True)
encoded_labels=np.asarray(encoded)

img_rows=224
img_cols=224
no_channels=3
feature_x=[]
feature_y=[]
i=0
for f in tqdm(train_info['image_name'].values): # f for format ,jpg
    train_img = cv2.imread('scene_images/image_data/{}'.format(f),0)
    labels = encoded_labels[i]
    resized_imgs = cv2.resize(train_img, (img_rows, img_cols)) 
    feature_x.append(resized_imgs)
    feature_y.append(labels)
    i += 1
feature_x=np.expand_dims(feature_x,axis=3)


test_feature=[]
for f in tqdm(test_info['image_name'].values):
    test_img=cv2.imread('scene_images/image_data/{}'.format(f),0)
    resized_imgs=cv2.resize(test_img,(img_rows,img_cols))
    test_feature.append(resized_imgs)

test_data=np.expand_dims(test_feature,axis=3)

100%|██████████| 7301/7301 [00:03<00:00, 1909.89it/s]


In [0]:
from keras.layers import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(128,128,1)))
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling2D((2, 2),padding='same'))
model1.add(Dropout(0.25))
model1.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model1.add(Dropout(0.25))
model1.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model1.add(LeakyReLU(alpha=0.1))                  
model1.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model1.add(Dropout(0.25))
model1.add(Flatten())
model1.add(Dense(128, activation='linear'))
model1.add(LeakyReLU(alpha=0.1))           
model1.add(Dropout(0.25))
model1.add(Dense(6, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model1.compile(loss='categorical_crossentropy', optimizer=Adam(),metrics=['accuracy'])

In [0]:
from keras.callbacks import EarlyStopping
train_datagen=ImageDataGenerator( featurewise_center=True,featurewise_std_normalization=True,rotation_range=30, width_shift_range=0.2, height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')

train_datagen.fit(feature_x)
model1.fit_generator(train_datagen.flow(feature_x,feature_y, batch_size=32),steps_per_epoch=len(feature_x) / 32, epochs=100,callbacks=[EarlyStopping(monitor='acc',patience=4)])

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
533/532 [==============================] - 43s 81ms/step - loss: 1.1634 - acc: 0.5396
Epoch 2/100
533/532 [==============================] - 39s 72ms/step - loss: 0.9160 - acc: 0.6480
Epoch 3/100
533/532 [==============================] - 39s 73ms/step - loss: 0.8283 - acc: 0.6911
Epoch 4/100
533/532 [==============================] - 39s 72ms/step - loss: 0.7708 - acc: 0.7124
Epoch 5/100
533/532 [==============================] - 39s 72ms/step - loss: 0.7547 - acc: 0.7221
Epoch 6/100
533/532 [==============================] - 39s 72ms/step - loss: 0.7148 - acc: 0.7346
Epoch 7/100
533/532 [==============================] - 39s 72ms/step - loss: 0.6958 - acc: 0.7407
Epoch 8/100
533/532 [==============================] - 38s 72ms/step - loss: 0.6857 - acc: 0.7420
Epoch 9/100
533/532 [==============================] - 39s 72ms/step - loss: 0.6584 - acc: 0.7601
Epoch 10/100
533/532 [==============================] - 39s 72ms/step 